$\displaystyle \min \sum_{i \in N} \sum_{j \in N} \sum_{s \in S} \sum_{t \in T} c_{ij} x_{ijst}$

Subject to:

$\displaystyle
x_{sjst} = 0, \quad  j \in N, s \in S, t \in T, t \neq 1,$

$\displaystyle
x_{ijs1} = 0, \quad  i \in N \backslash \{s\}, j \in N, s \in S,$

$\displaystyle
\sum_{i \in N} x_{ijst} = \sum_{i \in N} x_{jis,t+1}, \quad j \in C \cup \{w\}, s \in S, t \in T,$

$\displaystyle
\sum_{i \in N} \sum_{t \in T} x_{ijst} \leq 1, \quad j \in N, s \in S,$

$\displaystyle 
\sum_{j \in N} \sum_{t \in T} x_{jist} \geq d_{is} - \sum_{s' \in S, s' \neq s} z_{iss'}, \quad i \in C, s \in S,$

$\displaystyle
3z_{iss'} \leq \sum_{j \in C \cup \{s\}} \sum_{t \in T} x_{jwst} + \sum_{j \in C \cup \{s\}} \sum_{t \in T} x_{jws't} + y_{is'}, \quad i \in C, s \in S, s' \in S, s \neq s',$ 

$\displaystyle
\sum_{j \in C\cup \{s\}} \sum_{t \in T} t\ x_{jist} - \sum_{j \in C \cup \{s\} \cup \{w\}} \sum_{t \in T} t\ x_{jwst} \geq |T| (y_{is}-1), \quad i \in N, s \in S,$

$\displaystyle
y_{is} \leq \sum_{j \in C \cup \{s\}} \sum_{t \in T} x_{jist}, \quad i \in C, s \in S.$


In [11]:
using JuMP
using GLPKMathProgInterface
include("readData.jl") # Import function readData, used for reading input parameters

printSolution (generic function with 2 methods)

In [12]:
function defineModel(m, nodes, suppliers, consumers, demand, edgeExists)

    T = 1:length(nodes)  # Possible legs

    @variable(m, x[i=nodes, j=nodes, s=suppliers, t=T; edgeExists[s, i, j]], Bin)
    @variable(m, z[i=consumers, s=suppliers, sBar=suppliers; s != sBar], Bin)
    @variable(m, y[i=consumers, s=suppliers], Bin)

    @objective(m, Min, sum(cost[i, j]*x[i, j, s, t]
        for i in nodes, j in nodes, s in suppliers, t in T if edgeExists[s, i, j]))

    # We only leave suppliers at the first leg:
    @constraint(m, [j in nodes,
                    s in suppliers,
                    t in T; (t != 1) && (edgeExists[s, j, s])],
                    x[s, j, s, t] == 0)

    # We cannot leave any other node but suppliers in the first leg:
    @constraint(m, [i in nodes,
                    j in nodes,
                    s in suppliers; (i in suppliers) == false && (edgeExists[s, i, j])],
                    x[i, j, s, 1] == 0)

    # Flow balance constraints:
    @constraint(m, [s in suppliers, j in nodes, t in T[1:end-1]; j != s],
    sum(x[i, j, s, t] for i in nodes if edgeExists[s, i, j]) ==
    sum(x[j, i, s, t+1] for i in nodes if edgeExists[s, j, i]))

    # We only visit each node at most once with each supplier
    for s in suppliers, j in nodes
        @constraint(m, sum(x[i, j, s, t] for i in nodes, t in T if edgeExists[s, i, j]) <= 1)
    end

    # Demand constraints:
    @constraint(m, [i in consumers, s in suppliers],
        sum(x[j, i, s, t] for j in nodes, t in T if edgeExists[s, i, j]) >=
        demand[i, s] - sum(z[i, s, sBar] for sBar in suppliers if s != sBar))

    for i in consumers, s in suppliers
        @expression(m, sVisitsCD, sum(x[j, w, s, t] for j in nodes, t in T if edgeExists[s, j, w]))

        for sBar in suppliers
            if s != sBar
                @expression(m, sBarVisitsCD, 
                            sum(x[j, w, sBar, t] for j in nodes, t in T if edgeExists[sBar, j, w]))
                @constraint(m, 3*z[i, s, sBar] <= sVisitsCD + sBarVisitsCD + y[i, sBar])
            end
        end
    end

    @constraint(m, [i in consumers, s in suppliers],
        sum(t*x[j, i, s, t] for j in nodes, t in T if edgeExists[s, j, i]) -
        sum(t*x[j, w, s, t] for j in nodes, t in T if edgeExists[s, j, w]) >= length(T)*(y[i, s]-1))

    @constraint(m, [i in consumers, s in suppliers],
        y[i, s] <= sum(x[j, i, s, t] for j in nodes, t in T if edgeExists[s, j, i]))

    m, x, T
end

filename = "data1.csv"

suppliers, consumers, warehouse, nodes, posX, posY, cost, demand = readData(filename)
w = warehouse[1]

# After preprocessing we get the edgeExists map, 
# which tell us if a particular supplier, s, can traverse edge (i, j)
nodes, consumers, suppliers, edgeExists = preprocessing(nodes, suppliers, consumers, demand)

# Time Limit (tm_lim) is in mili-seconds:
m = Model(solver=GLPKSolverMIP(tm_lim=5000, msg_lev=2))

m, x, T = defineModel(m, nodes, suppliers, consumers, demand, edgeExists)

solve(m)

printSolution(m, x)

      0: obj =   0.000000000e+00 inf =   1.300e+01 (13)
    233: obj =   5.848602719e+02 inf =   1.662e-15 (0)
*   500: obj =   2.403272906e+02 inf =   2.721e-16 (759) 1
*   772: obj =   1.744989748e+02 inf =   0.000e+00 (0) 1
+   772: mip =     not found yet >=              -inf        (1; 0)
+  5239: >>>>>   4.896222297e+02 >=   2.427393813e+02  50.4% (162; 4)
+  5937: mip =   4.896222297e+02 >=   2.492331711e+02  49.1% (109; 130)
489.6222297251765
1 1 15
1 15 8
1 8 7
1 7 4
1 4 6
1 6 5
1 5 13
1 13 9
1 9 14
1 14 11
1 11 1
2 2 15
2 15 8
2 8 10
2 10 11
2 11 2



$$\sum_{x_{ijst} \in \mathcal{X}^-} x_{ijst} + \sum_{x_{ijst} \in \mathcal{X}^+} (1 - x_{ijst}) \leq k,$$

In [44]:
m = Model(solver=GLPKSolverMIP(tm_lim=50000, msg_lev=0))
# using Gurobi
# m = Model(solver=GurobiSolver(TimeLimit=8))

m, x, T = defineModel(m, nodes, suppliers, consumers, demand, edgeExists)

status = solve(m)

@constraint(m, localBranchConst,
                sum(x[i, j, s, t] for i in nodes, j in nodes, s in suppliers, t in T
                if edgeExists[s, i, j] && (getvalue(x[i, j, s, t])) == 0) + 
                sum(1 - x[i, j, s, t] for i in nodes, j in nodes, s in suppliers, t in T
                if edgeExists[s, i, j] && (getvalue(x[i, j, s, t]) == 1)) <= 10)

# nIter = 6
objectives = []
for i in 10:10:40
    @show getobjectivevalue(m)
    push!(objectives, getobjectivevalue(m))

    # Adding the "local branch" constraint:
#     @constraint(m, localBranchConst <= 10) 
    JuMP.setRHS(localBranchConst, i)
    solve(m)
#     @show localBranchConst
end
printSolution(m, x)
@show objectives

getobjectivevalue(m) = 395.42279860154736


getobjectivevalue(m) = 524.6804554840855


getobjectivevalue(m) = 552.16832012802


getobjectivevalue(m) = 552.16832012802


552.16832012802
1 1 15
1 15 13
1 13 14
1 14 9
1 9 10
1 10 11
1 11 8
1 8 7
1 7 4
1 4 5
1 5 6
1 6 1
2 2 8
2 8 15
2 15 2

objectives = Any[395.423,524.68,552.168,552.168]


4-element Array{Any,1}:
 395.423
 524.68 
 552.168
 552.168

In [54]:
m = Model(solver=GLPKSolverMIP(tm_lim=17000, msg_lev=0))
# using Gurobi
# m = Model(solver=GurobiSolver(TimeLimit=8))

m, x, T = defineModel(m, nodes, suppliers, consumers, demand, edgeExists)

status = solve(m)


@expression(m, orObj, sum(cost[i, j]*x[i, j, s, t]
        for i in nodes, j in nodes, s in suppliers, t in T if edgeExists[s, i, j]))

objectives = []
for i in 10:10:40
    @show getvalue(orObj)
    push!(objectives, getvalue(orObj))

    @objective(m, Min, 
                sum(x[i, j, s, t] for i in nodes, j in nodes, s in suppliers, t in T
                if edgeExists[s, i, j] && (getvalue(x[i, j, s, t])) == 0) + 
                sum(1 - x[i, j, s, t] for i in nodes, j in nodes, s in suppliers, t in T
                if edgeExists[s, i, j] && (getvalue(x[i, j, s, t]) == 1)) )

    # Adding the "local branch" constraint:
    @constraint(m, orObj <= getvalue(orObj) - 1)
                                                
    status = solve(m)
    if status == :Infeasible
        println("Could not find a feasible solution")
        break
    end
#     @show localBranchConst
end
printSolution(m, x)
@show objectives

getvalue(orObj) = 472.93525782229426


getvalue(orObj) = NaN


Excessive output truncated after 524295 bytes.WARNING: Variable value not defined for x[7,13,1,6]. Check that the model was properly solved.


4-element Array{Any,1}:
 472.935
 NaN    
 NaN    
 NaN    